In [1]:
import tensorflow as tf
import pandas as pd

In [32]:
'''هنا نسترد الملف'''
CSV_COLUMN_NAMES = ['buying', 'maint', 'doors', 'persons', 'lug_boot','safety','rating']
data = pd.read_csv("Car_Data.csv",names=CSV_COLUMN_NAMES, skiprows=1)
data.head()


,buying,maint,doors,persons,lug_boot,safety,rating
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [33]:
"""هنا نحول كل الكلام الي ارقام لكي يفهمها الموديل """
from sklearn import preprocessing

# Create a label (category) encoder object
le = preprocessing.LabelEncoder()
# Fit the encoder to the pandas column
data['buying'] = le.fit_transform(data['buying'])
data['maint'] = le.fit_transform(data['maint'])
data['doors'] = le.fit_transform(data['doors'])
data['persons'] = le.fit_transform(data['persons'])
data['lug_boot'] = le.fit_transform(data['lug_boot'])
data['safety'] = le.fit_transform(data['safety'])
data['rating'] = le.fit_transform(data['rating'])

In [10]:
"""
هنا نفصل البيانات بين 
x,y  

"""
data_y = data.pop('rating')


In [ ]:
data.head()

In [12]:
'''هنا نفصل بين البيانات الي بيانات للتدريب وبيانات للاختبار'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,data_y,test_size=0.3,random_state=101)

In [14]:
"""
هذه الدالة تحول كل 
x, y 
الي قواميس لكي يفهمها موديل 
DNNClassifier
"""
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


In [23]:
'''
هنا نضع كل الفيتشرز في
list
'''
# Feature columns describe how to use the input.
my_feature_columns = []
for key in data.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [24]:
'''
هنا نعطي موديل 
DNNClassifier 
كل المعطيات الي جهزناها 
'''
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[512, 256, 128],
    # The model must choose between 3 classes.
    n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\islam\\AppData\\Local\\Temp\\tmpio_nt0wn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
"""هنا ندرب الموديل"""
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(X_train, y_train, training=True),
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\islam\AppData\Local\Temp\tmpio_nt0wn\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.4506825, step = 0
INFO:tensorflow:global_step/sec: 35.8774
INFO:tensorflow:loss = 0.99311376, step = 100 (2.801 sec)
INFO:tensorflow:global_step/sec: 35.7867
INFO:tensorflow:loss = 0.86523306, step = 200 (2.797 sec)
INFO:tensorflow:global_step/sec: 35.6719
INFO:tensorflow:loss = 0.8408917, step = 300 (2.800 sec)
INFO:tensorflow:global_step/sec: 37.3874
INFO:tensorflow:loss = 0.7976711, step = 400 (2.682 sec)
INFO:tensorflow:global_step/sec: 34.476
INFO:tensorflow:loss = 0.7447855,

In [26]:
"""هنا يحسب نسبة دقة البيانات في بيانات الاختبار"""
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(X_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-11-02T11:42:48
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\islam\AppData\Local\Temp\tmpio_nt0wn\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 4.46020s
INFO:tensorflow:Finished evaluation at 2022-11-02-11:42:53
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7919075, average_loss = 0.5384158, global_step = 5000, loss = 0.42215455
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\islam\AppData\Local\Temp\tmpio_nt0wn\model.ckpt-5000

Test set accuracy: 0.792



In [27]:
"""هذه الدالة لكي تحسب القيم المتوقعة"""
# Generate predictions from the model
def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(X_test))

# المصدر الذي اقتبس منه صناعة هذا الكود
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/estimator/premade.ipynb